### • Business Understanding

•  **Introduction:** The project, involves working with an American retail chain operating across California (CA), Texas (TX), and Wisconsin (WI). This retailer offers a diverse range of products, including hobbies, foods, and household items, across ten stores. The objective is to develop and deploy two distinct machine learning models as APIs to address specific business challenges:

1. Predictive Model: A predictive model to forecast sales revenue for specific items in particular stores on given dates.
2. Forecasting Model: A time-series-based forecasting model to predict total sales revenue across all stores and items for the next seven days.

These models will help optimize inventory, pricing, and decision-making for our retail partner.

•  **Dataset:** To develop and evaluate the models, below datasets are provided:

        - Training Data
        - Evaluation Data
        - Calendar
        - Events
        - Items Price per Week

•  **Business Problem:** The primary business problems that the machine learning models aim to address are as follows:

1. Sales Prediction: The retailer needs to accurately predict the sales revenue for individual items in specific stores for any given date.

2. Sales Forecasting: The retailer seeks to forecast total sales revenue across all stores and items for the next seven days.

This information is crucial for inventory management, pricing strategies, and overall business planning. Our machine learning models aim to address these business challenges by providing precise predictions and forecasts.

The following activitives are performed for this Regression learning task.

• Business Understanding

• Data Understanding

        1] Loading Data
        2] Exploring Data
        3] Combining the training and other datasets

• Data Preparation

        4] Feature Engineering
        5] Features Selection
        6] Splitting Data into Different Sets

• Modeling

        7] Assessing Baseline Performance
        8] Sales Prediction Model: XGBoost Algorithm
        9] Sales Forecasting Model: Prophet Algorithm

• Model Evaluation

        10] Analysing Model Performance

In [1]:
# Importing Python and the necessary libraries
import numpy as np
import pandas as pd

# Importing formatting and other required libraries
import sys
import warnings

# Including the project root directory
sys.path.append('/Users/monalipatil/Monali/MDSI-Semester1/Advanced Machine Learning Application/Assignment2/adv_mla_assignment2')

# Importing functions defined to perform EDA and data processing tasks
from src.data.sets import checking_null_values, imputing_missing_values, extract_date_components

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
#Ignoring warnings to maintain a clean coding.
warnings.filterwarnings('ignore')

## Exploratory Data Analysis (EDA) and Data Preparation Activities.

### • Data Understanding

#### 1] Loading  Data

In [4]:
# Defining data files path 
# Note: Change this path to the relevant directory
file_url = '/Users/monalipatil/Monali/MDSI-Semester1/Advanced Machine Learning Application/Assignment2/adv_mla_assignment2'

# Loading the retail data into a separate pandas dataframe
df_train = pd.read_csv(file_url + '/data/raw/sales_train.csv')
df_calendar = pd.read_csv(file_url + '/data/raw/calendar.csv')
df_calendar_events = pd.read_csv(file_url + '/data/raw/calendar_events.csv')
df_sell_prices = pd.read_csv(file_url + '/data/raw/items_weekly_sell_prices.csv')

#### 2] Exploring Data

• Checking a some data points of the datasets.

In [5]:
# Checking the initial records of the dataset - training
df_train.head()

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,...,d_1532,d_1533,d_1534,d_1535,d_1536,d_1537,d_1538,d_1539,d_1540,d_1541
0,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,1,1,0,1,0,1,0,0,1
1,HOBBIES_1_002_CA_1_evaluation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
2,HOBBIES_1_003_CA_1_evaluation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
3,HOBBIES_1_004_CA_1_evaluation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,8,2,0,8,2,3,1,1,3,8
4,HOBBIES_1_005_CA_1_evaluation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,2,0,1,3,2,1,1,2,2,3


In [6]:
# Checking the initial records of the dataset - weekly selling prices of the item
df_sell_prices.head()

,store_id,item_id,wm_yr_wk,sell_price
0,CA_1,HOBBIES_1_001,11325,9.58
1,CA_1,HOBBIES_1_001,11326,9.58
2,CA_1,HOBBIES_1_001,11327,8.26
3,CA_1,HOBBIES_1_001,11328,8.26
4,CA_1,HOBBIES_1_001,11329,8.26


In [7]:
# Checking the initial records of the dataset - calendar
df_calendar.head()

,date,wm_yr_wk,d
0,2011-01-29,11101,d_1
1,2011-01-30,11101,d_2
2,2011-01-31,11101,d_3
3,2011-02-01,11101,d_4
4,2011-02-02,11101,d_5


In [8]:
# Checking the initial records of the dataset - calendar events
df_calendar_events.head()

,date,event_name,event_type
0,2011-02-06,SuperBowl,Sporting
1,2011-02-14,ValentinesDay,Cultural
2,2011-02-21,PresidentsDay,National
3,2011-03-09,LentStart,Religious
4,2011-03-16,LentWeek2,Religious


• Examining the dimension of the dataset.

In [9]:
# Checking the size of the datasets
print('Training Dataset:', df_train.shape)
print('Weekly Selling Prices of the Item Dataset:', df_sell_prices.shape)
print('Calander Dataset:', df_calendar.shape)
print('Calander Events Dataset:', df_calendar_events.shape)

Training Dataset: (30490, 1547)
Weekly Selling Prices of the Item Dataset: (6841121, 4)
Calander Dataset: (1969, 3)
Calander Events Dataset: (167, 3)


• Checking attributes names. 

In [10]:
# Viewing the attributes of the datasets
print('Training Dataset Attribute Names:', df_train.columns)
print('\n Weekly Selling Prices of the Item Dataset Attribute Names:', df_sell_prices.columns)
print('\n Calander Dataset Attribute Names:', df_calendar.columns)
print('\n Calander Events Dataset Attribute Names:', df_calendar_events.columns)

Training Dataset Attribute Names: Index(['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id', 'd_1',
       'd_2', 'd_3', 'd_4',
       ...
       'd_1532', 'd_1533', 'd_1534', 'd_1535', 'd_1536', 'd_1537', 'd_1538',
       'd_1539', 'd_1540', 'd_1541'],
      dtype='object', length=1547)

 Weekly Selling Prices of the Item Dataset Attribute Names: Index(['store_id', 'item_id', 'wm_yr_wk', 'sell_price'], dtype='object')

 Calander Dataset Attribute Names: Index(['date', 'wm_yr_wk', 'd'], dtype='object')

 Calander Events Dataset Attribute Names: Index(['date', 'event_name', 'event_type'], dtype='object')


• Inspecting for any null values in the datasets.

In [11]:
# Checking whether there are any null values in the dataset - training
print('Verifying the presence of null values in dataset - training')
checking_null_values(df_train)

Verifying the presence of null values in dataset - training
id          0
item_id     0
dept_id     0
cat_id      0
store_id    0
           ..
d_1537      0
d_1538      0
d_1539      0
d_1540      0
d_1541      0
Length: 1547, dtype: int64


In [12]:
# Checking whether there are any null values in the dataset - weekly selling prices of the item
print('Verifying the presence of null values in dataset - weekly selling prices of the item')
checking_null_values(df_sell_prices)

Verifying the presence of null values in dataset - weekly selling prices of the item
store_id      0
item_id       0
wm_yr_wk      0
sell_price    0
dtype: int64


In [13]:
# Checking whether there are any null values in the dataset - calendar
print('Verifying the presence of null values in dataset - calendar')
checking_null_values(df_calendar)

Verifying the presence of null values in dataset - calendar
date        0
wm_yr_wk    0
d           0
dtype: int64


In [14]:
# Checking whether there are any null values in the dataset - calendar events
print('Verifying the presence of null values in dataset - calendar events')
checking_null_values(df_calendar_events)

Verifying the presence of null values in dataset - calendar events
date          0
event_name    0
event_type    0
dtype: int64


#### 3] Combining the training and other datasets

• Reshaping the training dataset and renaming the attributes.

In [15]:
# Transforming the training dataset and remaining the attributes
df_train = df_train.melt(id_vars=['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'], var_name='dayofsale', value_name='volume')

# Reviewing the training dataset after reshaping
df_train.sample(3)

,id,item_id,dept_id,cat_id,store_id,state_id,dayofsale,volume
16042929,FOODS_2_314_CA_2_evaluation,FOODS_2_314,FOODS_2,FOODS,CA_2,CA,d_527,0
29296090,HOUSEHOLD_2_203_WI_2_evaluation,HOUSEHOLD_2_203,HOUSEHOLD_2,HOUSEHOLD,WI_2,WI,d_961,0
12078691,HOUSEHOLD_2_507_CA_2_evaluation,HOUSEHOLD_2_507,HOUSEHOLD_2,HOUSEHOLD,CA_2,CA,d_397,1


• Combining the training and calendar dataframes using the sales day information.

In [16]:
# Merging the training and calendar dataframes using the sales day information 
df_train_calender_merged = df_train.merge(df_calendar, left_on='dayofsale', right_on='d', how='left')

# Checking few records after combining the training and calendar dataframes
df_train_calender_merged.sample(3)

,id,item_id,dept_id,cat_id,store_id,state_id,dayofsale,volume,date,wm_yr_wk,d
38274168,HOBBIES_1_074_CA_4_evaluation,HOBBIES_1_074,HOBBIES_1,HOBBIES,CA_4,CA,d_1256,1,2014-07-07,11423,d_1256
1004670,HOUSEHOLD_2_454_WI_3_evaluation,HOUSEHOLD_2_454,HOUSEHOLD_2,HOUSEHOLD,WI_3,WI,d_33,0,2011-03-02,11105,d_33
31609449,HOBBIES_2_051_WI_1_evaluation,HOBBIES_2_051,HOBBIES_2,HOBBIES,WI_1,WI,d_1037,0,2013-11-30,11345,d_1037


• Combining the training and calendar events dataframes based on the date information.

In [17]:
# Merging the training and calendar events dataframes based on the date information 
df_train_calender_events_merged = df_train_calender_merged.merge(df_calendar_events, left_on='date', right_on='date', how='left')

# Checking few records after combining the training and calendar events dataframes
df_train_calender_events_merged.sample(3)

,id,item_id,dept_id,cat_id,store_id,state_id,dayofsale,volume,date,wm_yr_wk,d,event_name,event_type
30227042,FOODS_3_081_CA_4_evaluation,FOODS_3_081,FOODS_3,FOODS,CA_4,CA,d_990,0,2013-10-14,11338,d_990,ColumbusDay,National
31143452,HOUSEHOLD_1_411_TX_1_evaluation,HOUSEHOLD_1_411,HOUSEHOLD_1,HOUSEHOLD,TX_1,TX,d_1020,0,2013-11-13,11342,d_1020,NaN,NaN
38232410,HOUSEHOLD_1_444_WI_3_evaluation,HOUSEHOLD_1_444,HOUSEHOLD_1,HOUSEHOLD,WI_3,WI,d_1250,0,2014-07-01,11422,d_1250,NaN,NaN


• Combining the training and sell prices dataframes based on the common attributes.

In [18]:
# Merging the training and sell prices dataframes based on the relavant information
df_train_final = df_train_calender_events_merged.merge(df_sell_prices, left_on=['store_id', 'item_id', 'wm_yr_wk'], 
                                                                       right_on=['store_id','item_id','wm_yr_wk'], 
                                                                       how='left')

# Checking few records after combining the training and sell prices dataframes
df_train_final.sample(3)

,id,item_id,dept_id,cat_id,store_id,state_id,dayofsale,volume,date,wm_yr_wk,d,event_name,event_type,sell_price
22602702,HOBBIES_2_050_CA_4_evaluation,HOBBIES_2_050,HOBBIES_2,HOBBIES,CA_4,CA,d_741,0,2013-02-07,11302,d_741,NaN,NaN,1.97
3503255,FOODS_3_782_WI_2_evaluation,FOODS_3_782,FOODS_3,FOODS,WI_2,WI,d_114,0,2011-05-22,11117,d_114,NaN,NaN,2.48
20917740,HOUSEHOLD_2_505_CA_1_evaluation,HOUSEHOLD_2_505,HOUSEHOLD_2,HOUSEHOLD,CA_1,CA,d_686,1,2012-12-14,11246,d_686,NaN,NaN,4.97


### • Data Preparation

#### 4] Feature Engineering

• Determining the income generated from item sales and the item's weekly selling price.

In [19]:
# Calculating the revenue from the items sold and the item's weekly selling price
df_train_final['revenue'] = df_train_final['volume'] * df_train_final['sell_price']
df_train_final.sample(3)

,id,item_id,dept_id,cat_id,store_id,state_id,dayofsale,volume,date,wm_yr_wk,d,event_name,event_type,sell_price,revenue
39902738,HOBBIES_2_060_WI_1_evaluation,HOBBIES_2_060,HOBBIES_2,HOBBIES,WI_1,WI,d_1305,0,2014-08-25,11430,d_1305,NaN,NaN,0.97,0.00
17895153,HOUSEHOLD_1_008_WI_3_evaluation,HOUSEHOLD_1_008,HOUSEHOLD_1,HOUSEHOLD,WI_3,WI,d_586,3,2012-09-05,11232,d_586,NaN,NaN,5.47,16.41
135893,FOODS_1_128_TX_1_evaluation,FOODS_1_128,FOODS_1,FOODS,TX_1,TX,d_5,1,2011-02-02,11101,d_5,NaN,NaN,3.18,3.18


• Checking the summary overview of the dataset.

In [20]:
# Checking the summary information of the attributes of the dataset
df_train_final.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 47107050 entries, 0 to 47107049
Data columns (total 15 columns):
 #   Column      Dtype  
---  ------      -----  
 0   id          object 
 1   item_id     object 
 2   dept_id     object 
 3   cat_id      object 
 4   store_id    object 
 5   state_id    object 
 6   dayofsale   object 
 7   volume      int64  
 8   date        object 
 9   wm_yr_wk    int64  
 10  d           object 
 11  event_name  object 
 12  event_type  object 
 13  sell_price  float64
 14  revenue     float64
dtypes: float64(2), int64(2), object(11)
memory usage: 5.6+ GB


• Checking summary statistics of the dataset.

In [21]:
# Inspecting statistical summary information for all dataset attributes of different data types
df_train_final.describe(include='all')

,id,item_id,dept_id,cat_id,store_id,state_id,dayofsale,volume,date,wm_yr_wk,d,event_name,event_type,sell_price,revenue
count,47107050,47107050,47107050,47107050,47107050,47107050,47107050,4.710705e+07,47107050,4.710705e+07,47107050,3963700,3963700,3.481517e+07,3.481517e+07
unique,30490,3049,7,3,10,3,1541,NaN,1541,NaN,1541,30,4,NaN,NaN
top,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1_001,FOODS_3,FOODS,CA_1,CA,d_86,NaN,2011-04-24,NaN,d_86,SuperBowl,Religious,NaN,NaN
freq,1545,15450,12715350,22201650,4710705,18842820,60980,NaN,60980,NaN,60980,152450,1372050,NaN,NaN
mean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.087059e+00,NaN,1.128854e+04,NaN,NaN,NaN,4.383812e+00,4.102582e+00
std,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.934360e+00,NaN,1.207423e+02,NaN,NaN,NaN,3.358256e+00,1.046314e+01
min,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000e+00,NaN,1.110100e+04,NaN,NaN,NaN,1.000000e-02,0.000000e+00
25%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000e+00,NaN,1.120400e+04,NaN,NaN,NaN,2.080000e+00,0.000000e+00
50%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000e+00,NaN,1.130700e+04,NaN,NaN,NaN,3.440000e+00,0.000000e+00
75%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.000000e+00,NaN,1.140900e+04,NaN,NaN,NaN,5.840000e+00,4.850000e+00


#### 5] Features Selection

• Selecting the appropriate attributes.

In [22]:
# Creating a copy of the df_train_final and storing it in a dataframe named df_retailer_data
df_retailer_data = df_train_final.copy()

In [23]:
# Selecting the relavant features
df_selected_features = ['item_id', 'store_id', 'date', 'event_name', 'event_type', 'revenue']
df_retailer_data = df_retailer_data[df_selected_features]

• Inspecting if any missing values in the selected features.

In [24]:
# Checking whether there are any missing values in the dataset.
df_retailer_data.isna().sum()

item_id              0
store_id             0
date                 0
event_name    43143350
event_type    43143350
revenue       12291876
dtype: int64

• Replacing missing values in the features 'event_name' and 'event_type' with None.

In [25]:
# Invoking the function for the 'event_name' column
imputing_missing_values(df_retailer_data, 'event_name')
imputing_missing_values(df_retailer_data, 'event_type')

• Replacing missing values in the target 'revenue' feature with 0.

In [26]:
# Invoking the function for the 'revenue' column
imputing_missing_values(df_retailer_data, 'revenue')

• Verifying the missing values in the dataset following imputing process.

In [27]:
# Verifying the missing values in the dataset post imputing process
df_retailer_data.isna().sum()

item_id       0
store_id      0
date          0
event_name    0
event_type    0
revenue       0
dtype: int64

#### 6] Splitting Data into Different Sets

• Dividing the retailer data into training sets (from 2011 to 2014) and validation sets (from 2014 to 2016).

In [28]:
# Converting the 'date' attribute to a datetime datatype
df_retailer_data['date'] = pd.to_datetime(df_retailer_data['date'])

In [29]:
# Spliting the retailer data into training (spanning from 2011 to 2014) and validation (covering the period from 2014 to 2016) datasets
df_training = df_retailer_data[(df_retailer_data['date'] >= '2011-01-01') & (df_retailer_data['date'] <= '2014-12-31')]
df_validation = df_retailer_data[(df_retailer_data['date'] >= '2015-01-01') & (df_retailer_data['date'] <= '2016-12-31')]

• Deriving different date-related attributes from the date feature.

In [30]:
# Invoking the function to extract date components
# Extracting the day of the week, month, year, and week of the year information
extract_date_components(df_training, 'date')
extract_date_components(df_validation, 'date')

• Verifying dimensions and attribute names of the datasets following the splitting process.

In [31]:
# Confirming the dimensions of the datasets post splitting
print('Training dataset:', df_training.shape)
print('Validation dataset:', df_validation.shape)

Training dataset: (43814130, 10)
Validation dataset: (3292920, 10)


In [32]:
# Checking the attributes names of the datasets post splitting
print('Training dataset:', df_training.columns)
print('Validation dataset:', df_validation.columns)

Training dataset: Index(['item_id', 'store_id', 'date', 'event_name', 'event_type', 'revenue',
       'day_of_week', 'month', 'year', 'week_of_year'],
      dtype='object')
Validation dataset: Index(['item_id', 'store_id', 'date', 'event_name', 'event_type', 'revenue',
       'day_of_week', 'month', 'year', 'week_of_year'],
      dtype='object')


• Saving the training and validation datasets to a CSV file.

In [33]:
# Loading the training and validation datasets to the csv file
df_training.to_csv(file_url + '/data/processed/retail_training_dataset.csv', index=False)
df_validation.to_csv(file_url + '/data/processed/retail_validation_dataset.csv', index=False)